Example of the Empirical Variability Model class

In [3]:
import sys
sys.path.append("..")
import ensvar as var
import astropy.io.fits as fits
import numpy

The model depends on an input mock AGN catalogue. This can be generated e.g. by adoptinbg methods described in [Georgakakis et al. 2020](https://ui.adsabs.harvard.edu/abs/2020MNRAS.499..710G/abstract). The minimum number of columns of the input mock AGN table is redshift (assumed name 'Z'), stellar mass (assumed name 'LGM') and X-ray luminosity in the 2-10keV band (assumed name 'LGLX'). The catalogue is assumed to be an instance of numpy record arrays. The data read from fits files via the  astropy.io.fits module is a special subclass of numpy.recarray. In the example below the data can be read from a fits table or provided as numpy record array. 

In [4]:
'''
example of reading a mock AGN catalogue in fits format
'''
hdu = fits.open("../DATA/SIM_AIRD15_VAR_PHOT.fits")
data=hdu[1].data
hdu.close()

'''
example of directly providing an numpy record array with the minimum
number of information required by the Empirical Variability Model class, 
i.e. redshift ('Z'), Stellar mass ('LGM') and 2-10 keV X-ray 
luminosity ('LGLX')
'''
data = numpy.rec.array([(0.5,11.5, 43.0),
                    (0.7, 10.5, 42.6),
                    (0.8, 12.4, 43.9)],
                    formats='float64,float64,float64',
                    names='Z,LGM,LGLX')
    

Define the Empirical Variability Model class, providing as input:

* the mock AGN catalogue, 
* the redshift range (minimum and maximum redshift) of the sample. The `ZMIN` and `ZMAX` parameters are used to filter the input catalogue of mock AGN to the specified redshifgt interval. 
* the PSD model, one of `Model1`, `Model2`, `Model3`, `Model4`. These four models correspond those proposed in [Paolillo et al. 2017] https://ui.adsabs.harvard.edu/abs/2017MNRAS.471.4398P/abstract).
* the Black hole-mass vs stellar mass scaling realtion. These are define as functions in the ensvar.py module and include the (i) [Shankar et al. (2017)](https://ui.adsabs.harvard.edu/abs/2017ApJ...840...34S/abstract) "unbiased" relation (`GETMBHMSTAR_SHANKAR_SIGMA`) and (ii) the  [Shankar et al. (2020)](https://ui.adsabs.harvard.edu/abs/2020NatAs...4..282S/abstract) scaling relation based on the dynamically derived black-hole masses estimated by [Savorgnan et al. (2016)](https://ui.adsabs.harvard.edu/abs/2016ApJ...817...21S/abstract),  (`GETMBHMSTAR_SAV_SIGMA`). Both relations above include statter. Other scaling relations are also available (e.g. a simple linear parametrisation of the form $\log M_{BH} =  A + B \cdot (\log M_{star} - 11)$, where $A$, $B$ can be defined.
* and the function that estimates the Eddington ratio of mock AGN given 2-10keV luminosities and black-hole masses. The bolometric correction is relevant to this calculation. In the current implementation the function   `GETLGLEDD` of the `ensvar.py` module assumes a simple bolometric correction $L_{BOL} = 25 \cdot L_X(2-10\rm \,keV)$

The Empical Model (`EmpMo`) class method `updateSTRUCT` will generate all the relevant quantities that are needed for the variability calculation, e.g. black-hole mass and Eddington ratio.

In [5]:
EM=var.EmpMo(data=data, ZMIN=0.4, ZMAX=4.0, PSDfun="Model1", MMfun=var.GETMBHMSTAR_SHANKAR_SIGMA, LEDDfun=var.GETLGLEDD)
EM.updateSTRUCT()

the attribute `DSTRUCT` (python dictionary) holds all the important quantities (numpy arrays) relevant to the empirical variability model:

* `DSTRUCT["Z"]`: mock AGN redshift
* `DSTRUCT["WEIGHT"]`: mock AGN weight. By default one but can be modified to emulate an observational selection function. 
* `DSTRUCT["LGM"]`: mock AGN stellar mass (log10)
* `DSTRUCT["LGLX"]`: mock AGN X-ray luminosity (log10)
* `DSTRUCT["LGMBH"]`: mock AGN black hole mass (log10)
* `DSTRUCT["LGLEDD"]`: mock AGN Eddington ratio (log10)
* `DSTRUCT["PSDNORM"]`: normalisation of the PSD model (in log10)
* `DSTRUCT["NUB"]`: break frequencty of the PSD model (log10)
* `DSTRUCT["SIGMA2M"]`: model excess variance 
* `DSTRUCT["SIGMA2O"]`: observed excess variance, after applying to SIGMA2M the bias factor  $C \cdot 0.48^{\beta-1}$ [(Allevato et al. 2013)](https://ui.adsabs.harvard.edu/abs/2013ApJ...771....9A/abstract) 

In [6]:
for key in EM.DSTRUCT.keys():
    print(key)

print(EM.DSTRUCT['LGMBH'])

Z
WEIGHT
LGM
LGLX
LGMBH
LGLEDD
NUB
PSDNORM
SIGMA2O
SIGMA2M
[ 7.91190804  6.25033084 10.25955448]


The determimation of the excess variance requires a minimum and maximum timescale over which the variability is observed. These are defined in days at the observer frame. The method sigma2 of the Emprical Model Class estimates the excess variance. The `EmpMo` class also includes the attribiutes `biasC` and `biasBETA`, which define the  bias factor $C \cdot 0.48^{\beta-1}$  [Allevato et al. 2013](https://ui.adsabs.harvard.edu/abs/2013ApJ...771....9A/abstract). This has to do with observational biases in measurements of the excess variance e.g. because of the cadence of the observations. The default values are $C=1.3$ and $\beta=1.1$ but can be redefined.

In [7]:
DMINOBS=0.25
DMAXOBS=6205
EM.sigma2(DMINOBS, DMAXOBS)
print(EM.DSTRUCT['SIGMA2O'])

EM.biasC = 1.1
EM.biasBETA = 1.3
EM.sigma2(DMINOBS, DMAXOBS)
print(EM.DSTRUCT['SIGMA2O'])

[0.12818806 0.16395845 0.03892825]
[0.17544864 0.22440692 0.05328038]
